In [23]:
import numpy as np
from numba import jit

In [24]:
@jit(nopython = True)
def init_wave_func(k0_prime, N, n_cicles):
    psi = np.zeros((n_cicles, N), dtype = np.cdouble)
    i = 1
    while(i < N-1):
        psi[0, i] = np.exp(1j*k0_prime*i)*np.exp((-8*(4*i-N)**2)/N**2)
        i += 1
    return psi

@jit(nopython = True)
def init_potential(k0_prime, lamb, N):
    V_prime = np.zeros(N)
    i = int(2*N/5)
    while(i < 3*N/5):
        V_prime[i] = lamb*k0_prime**2
        i += 1
    return V_prime

@jit(nopython = True)
def compute_norm(psi, n):
    return np.linalg.norm(psi[n])
    
@jit(nopython = True)
def compute_alpha(V_prime, s_prime, N):
    alpha = np.zeros(N, dtype = np.cdouble)
    i = N-1
    while(i > 0):
        alpha[i-1] = -1/(2+(2j/s_prime)-V_prime[i]+alpha[i])
        i -= 1
    return alpha

@jit(nopython = True)
def compute_beta(psi, alpha, V_prime, s_prime, N):
    b = np.zeros(N, dtype = np.cdouble)
    beta = np.zeros(N, dtype = np.cdouble)
    i = 0
    while(i < N):
        b[i] = 4j*psi[0][i]/s_prime
        i +=1
    i = N-1
    while(i > 0):
        beta[i-1] = (b[i]-beta[i])/(2+(2j/s_prime)-V_prime[i]+alpha[i])
        i -= 1  
    return beta     

@jit(nopython = True)
def compute_chi(alpha, beta, N):
    chi = np.zeros(N, dtype = np.cdouble)
    i = 0
    while(i < N-1):
        chi[i+1] = alpha[i]*chi[i] + beta[i]
        i += 1
    return chi

@jit(nopython = True)
def compute_wave_func(psi, chi, n, N):
    psi[n+1] = chi - psi[n]
    return psi

def iterative_process(n_cicles, N, s, h, lamb):
    s_prime = s/h**2
    k0_prime = 2*np.pi*2/N

    psi = init_wave_func(k0_prime, N, n_cicles)
    V_prime = init_potential(k0_prime, lamb, N)
    alpha = compute_alpha(V_prime, s_prime, N)

    norm = np.zeros(n_cicles)
    norm[0] = compute_norm(psi, 0)
    i = 0
    while(i < n_cicles-1):
        beta = compute_beta(psi, alpha, V_prime, s_prime, N)
        chi = compute_chi(alpha, beta, N)
        psi = compute_wave_func(psi, chi, i, N)
        norm[i+1] = compute_norm(psi, i+1)
        i += 1
    return psi, norm


In [25]:
N= 50
n_cicles = 10
s = 0.01
h = 0.01
lamb = 0.3

psi, norm = iterative_process(n_cicles, N, s, h, lamb)
print(norm)
print(psi)

[2.35349063 2.35349063 2.35349063 2.35349063 2.35349063 2.35349063
 2.35349063 2.35349063 2.35349063 2.35349063]
[[ 0.00000000e+00+0.00000000e+00j  1.11030455e-03+2.85077755e-04j
   3.09849380e-03+1.70341138e-03j  7.17663562e-03+6.73930942e-03j
   1.32581302e-02+2.08914703e-02j  1.73465957e-02+5.33873320e-02j
   7.21818906e-03+1.14729830e-01j -3.98190586e-02+2.08738815e-01j
  -1.50976035e-01+3.20840406e-01j -3.40438686e-01+4.11519679e-01j
  -5.87466911e-01+4.26819695e-01j -8.28605543e-01+3.28068143e-01j
  -9.79496562e-01+1.23739192e-01j -9.79496562e-01-1.23739192e-01j
  -8.28605543e-01-3.28068143e-01j -5.87466911e-01-4.26819695e-01j
  -3.40438686e-01-4.11519679e-01j -1.50976035e-01-3.20840406e-01j
  -3.98190586e-02-2.08738815e-01j  7.21818906e-03-1.14729830e-01j
   1.73465957e-02-5.33873320e-02j  1.32581302e-02-2.08914703e-02j
   7.17663562e-03-6.73930942e-03j  3.09849380e-03-1.70341138e-03j
   1.11030455e-03-2.85077755e-04j  3.35462628e-04+2.15786020e-19j
   8.58318049e-05+2.20378618e